In [1]:
%%time
from algoutils import *

raw_courses = read('data_cleaning/courses.txt')
raw_industries = read('data_cleaning/industries.txt')
raw_job_industries = read('data_cleaning/job_industries.txt')

CPU times: user 8 ms, sys: 0 ns, total: 8 ms
Wall time: 376 ms


In [2]:
%%time
import cleaning
reload(cleaning)
# note: enchant module and library should be installed
from cleaning import clean

clean_courses = clean(raw_courses, style='course')
clean_industries = clean(raw_industries, style='industry')
clean_job_industries = clean(raw_job_industries, style='job title')

CPU times: user 17.2 s, sys: 264 ms, total: 17.5 s
Wall time: 21.3 s


In [3]:
%%time
# note: nltk module and nltk.wordnet corpus should be installed
from clustering import stem_cluster
 
course_clusters = stem_cluster(clean_courses)
industry_clusters = stem_cluster(clean_industries)
job_industry_clusters = stem_cluster(clean_job_industries, mode=5)

CPU times: user 68 ms, sys: 12 ms, total: 80 ms
Wall time: 97.9 ms


In [4]:
sorted(course_clusters)[:10]

['ACCOUNTANCY',
 'ACCOUNTING',
 'ADMINISTRATION',
 'ADVERTISING',
 'AGRIBUSINESS',
 'AGRICULTURAL',
 'AGRICULTURE',
 'ANIMATION',
 'APPLICATION',
 'APPLIED']

In [5]:
sorted(industry_clusters)[:10]

['ACCOUNTING',
 'ADVERTISING',
 'AGENCY',
 'ART',
 'ASSISTANT',
 'AUTOMOTIVE',
 'BANKING',
 'BEAUTY',
 'BEVERAGE',
 'BPO']

In [6]:
sorted(job_industry_clusters)[:10]

['BUSINESS',
 'CALL',
 'CENTER',
 'COMPUTER',
 'CONSTRUCTION',
 'DEPARTMENT',
 'DESIGN',
 'DEVELOPMENT',
 'DISTRIBUTION',
 'EDUCATION']

In [7]:
%%time

from analysis import LikelihoodMatrix

# note: pandas and sklearn modules should be installed
# initializes a likelihood data frame based on row and column labels.
# Performs a content-based filtering technique using features.
# vectorizer can be 'tfidf' or 'count'

CourseIndustry = LikelihoodMatrix(course_clusters, industry_clusters, vectorizer='tfidf', ngram_range=(3,4))

ImportError: cannot import name check_arrays

In [ ]:
CourseIndustry.dataframe.head()

In [ ]:
%%time

# note: pandas and sklearn modules should be installed
# derives the change in likelihood if an entry

# computes delta likelihood using the following steps: (non-verbatim)
# clabels = df.column_labels
# rlabels = df.row_labels
# vr = [cos_similarity(ur, label) for label in rlabels]
# vc = [cos_similarity(vr, label) for label in clabels]
# dL = vr * vc^T

d1 = CourseIndustry.delta('CALLCENTE', 'BUSINESS')
d2 = CourseIndustry.delta('MARKETING', 'RESEARCH & DEVELOPMENT')
d3 = CourseIndustry.delta('AGRICULTURE', 'FINANCE')

In [ ]:
%%time

# dynamic update of likelihood matrix
# just accumulate small likelihoods
# accounts for relative likelihood and not absolute percentage

users = [
    ('COMPUTER SCIENCE', 'INFORMATION TECHNOLOGY'),
    ('ACCOUNTING', 'BUSINESS'),
    ('BUSINESS ADMINSTRATION', 'FINANCE'),
    ('MARKETING', 'FINANCE'),
    ('MATHEMATICS', 'FINANCE'),
    ('SOCIAL SCIENCE', 'CONSULATION'),
    ('LITERATURE', 'WRITING'),
    ('MANAGEMENT', 'BANKING'),
    ('SCIENCES', 'RESEARCH')
]

for course, industry in users:
    CourseIndustry.add_match(course, industry)

In [ ]:
# get specific likelihood coefficient of a pair of clusters
# e.g. likelihood of courses from the BUSINESS industry

# note: 1.00 does not mean 100%
# likelihood metric is relative
# the more data, the more accurate the prediction

%time CourseIndustry.dataframe.BUSINESS.nlargest(10)

In [ ]:
%%time

# transpose data frame for rows
CourseIndustry.dataframe.T.SCIENCE.nlargest(6)

In [ ]:
%%time

# add count from a specific matching pair
# note: this is synonymous to CourseIndustry.dataframe += CourseIndustry.delta('PHYSICS', 'REPAIR SUPPLY')

CourseIndustry.add_match('PHYSICS', 'RESEARCH')

In [ ]:
%%time

# returns a vector of recommend likelihood of a course from a specific industry
# since courses are rows, we find column matches
# we can also make the method output percentages

matches = CourseIndustry.find_column_matches('PHYSICS', with_labels = True, percentage = True)

In [ ]:
# rank recommendations with the highest percentage
matches.nlargest(5)

In [ ]:
%%time

# we can also get the recommendation score of a specific pair
# this is equivalent to SV1^T * LM * SV2^T
# where SV is similarity_vector, LM is likelihood matrix

CourseIndustry.recommendation_score('PHYSICS', 'SCIENTIFIC RESEARCH')

In [ ]:
%%time

# You can also get the dot product of data frames
# to get bridged likelihood

IndustryJob = LikelihoodMatrix(industry_clusters, job_industry_clusters, vectorizer='tfidf', ngram_range=(3,4))
CourseJob = CourseIndustry * IndustryJob

In [ ]:
CourseJob.recommendation_score('PHYSICS', 'MANAGEMENT')

In [ ]:
CourseJob.recommendation_score('COMPUTER SCIENCE', 'ARCHITECTURE')

In [ ]:
CourseJob.recommendation_score('COMPUTER SCIENCE', 'TECHNOLOGY')

In [ ]:
# after adding a match to course and a job, recommendation score should go up
CourseJob.add_match('COMPUTER SCIENCE', 'TECHNOLOGY')
CourseJob.add_match('PROGRAMMING', 'COMPUTER')

In [ ]:
CourseJob.recommendation_score('PROGRAMMING', 'COMPUTER')